# Part 1: Setup + Document Loading

**Goal**: Load DataFlow's enterprise documents for RAG system
 

In [54]:
# Imports and setup
from pathlib import Path
from typing import List
from langchain.document_loaders import TextLoader, CSVLoader, JSONLoader, UnstructuredMarkdownLoader
from langchain.schema import Document

# Path to enterprise documents
#KNOWLEDGE_BASE_PATH = Path("Section_6_Real_World_RAG_Engineering/enterprise_knowledge_base")
KNOWLEDGE_BASE_PATH = Path("enterprise_knowledge_base")
print(f"📂 Knowledge Base Path: {KNOWLEDGE_BASE_PATH.resolve()}")
print("✅ Setup complete!")


📂 Knowledge Base Path: /Users/krishnaashili/git/pragatidev/AIAgentsBootcamp-master/Section_6_Real_World_RAG_Engineering/enterprise_knowledge_base
✅ Setup complete!


## Load All Documents

In [55]:
def load_enterprise_documents(base_path: Path) -> List[Document]:
    """Load all documents recursively with proper metadata"""
    
    all_docs = []
    
    print("🔄 Loading DataFlow's documents...")
    
    # Process each department folder
    for dept_path in base_path.iterdir():
        if not dept_path.is_dir():
            continue
            
        department = dept_path.name
        print(f"📁 {department}...")
        
        # Get ALL files recursively
        files = [f for f in dept_path.rglob("*") if f.is_file()]
        
        for file_path in files:
            try:
                # Choose loader by extension
                ext = file_path.suffix.lower()
                if ext == '.csv':
                    loader = CSVLoader(str(file_path))
                elif ext == '.json':
                    loader = JSONLoader(str(file_path), jq_schema='.', text_content=False)
                elif ext == '.md':
                    loader = UnstructuredMarkdownLoader(str(file_path))
                else:
                    loader = TextLoader(str(file_path), encoding='utf-8')
                
                # Load and add metadata
                docs = loader.load()
                for doc in docs:
                    doc.metadata.update({
                        "department": department,
                        "source_file": file_path.name,
                        "file_type": ext
                    })
                
                all_docs.extend(docs)
                rel_path = file_path.relative_to(dept_path)
                print(f"   ✅ {rel_path}")
                
            except Exception as e:
                print(f"   ❌ {file_path.name}: {str(e)[:30]}...")
    
    # Quick summary
    departments = set(doc.metadata['department'] for doc in all_docs)
    total_chars = sum(len(doc.page_content) for doc in all_docs)
    
    print(f"\n🎯 LOADED: {len(all_docs)} documents from {len(departments)} departments")
    print(f"📊 Content: {total_chars:,} characters")
    print(f"🏢 Departments: {', '.join(sorted(departments))}")
    
    return all_docs

# Load all documents
documents = load_enterprise_documents(KNOWLEDGE_BASE_PATH)

🔄 Loading DataFlow's documents...
📁 business_data...
   ✅ integration_partners.csv
   ✅ customer_analytics.csv
   ✅ billing_and_pricing.csv
📁 internal_operations...
   ✅ support_operations/customer_support_procedures.markdown
   ✅ support_operations/system_architecture.markdown
   ✅ sales_marketing/sales_playbook.json
   ✅ hr_policies/onboarding_checklist.json
   ✅ hr_policies/employee_handbook.txt
   ✅ product_releases/release_notes.json
📁 customer_facing...
   ✅ api_documentation.json
   ✅ troubleshooting_guide.txt
   ✅ terms_of_service.markdown
   ✅ product_user_guide.markdown
   ✅ competitive_analysis.txt
📁 legal_compliance...
   ✅ compliance_certifications.csv
   ✅ security_policies.txt
   ✅ terms_of_service.markdown
   ✅ privacy_policy.txt

🎯 LOADED: 212 documents from 4 departments
📊 Content: 262,596 characters
🏢 Departments: business_data, customer_facing, internal_operations, legal_compliance


## Validation

In [56]:
# Quick validation
print("🔍 Validation:")
print(f"   Documents: {len(documents)}  ")
print(f"   Departments: {len(set(doc.metadata['department'] for doc in documents))}  ")
print(f"   Content: {sum(len(doc.page_content) for doc in documents):,}  ")

if len(documents) >= 15:
    print("\n✅ SUCCESS! Ready for Part 2: Text Chunking")
else:
    print("\n⚠️ Low document count - check folder structure")

🔍 Validation:
   Documents: 212  
   Departments: 4  
   Content: 262,596  

✅ SUCCESS! Ready for Part 2: Text Chunking


## Part 1 Complete!

**✅ You've loaded DataFlow's complete knowledge base**

**Ready for Part 2:** Text chunking (the critical RAG skill)
 

# Part 2: Text Chunking

**Goal**: Transform 212 documents into optimally-sized chunks for RAG
 

In [57]:
# Imports for text chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List
from langchain.schema import Document

print("✅ Text chunking tools imported!")
print(f"📄 Starting with {len(documents)} documents from Part 1")

✅ Text chunking tools imported!
📄 Starting with 212 documents from Part 1


## Smart Chunking Strategy

**Industry Best Practice**: 1000 characters with 200 overlap
- **Why 1000 chars?** Perfect balance for embedding models
- **Why 200 overlap?** Preserves context across chunks
- **Recursive splitting**: Tries sentences, then words, then characters

In [58]:
def create_smart_chunks(documents: List[Document]) -> List[Document]:
    """Split documents into optimal chunks for RAG"""
    
    print("✂️ Creating smart chunks...")
    
    # Industry-standard chunking settings
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,        # Optimal for embedding models
        chunk_overlap=200,      # Preserve context
        length_function=len,    # Character-based
        separators=[            # Try these in order:
            "\n\n",              # Paragraphs first
            "\n",                # Then lines
            ". ",                # Then sentences
            " ",                 # Then words
            "",                  # Finally characters
        ]
    )
    
    all_chunks = []
    stats = {
        "original_docs": len(documents),
        "total_chunks": 0,
        "by_department": {},
        "by_file_type": {}
    }
    
    # Process each department
    for dept in set(doc.metadata['department'] for doc in documents):
        dept_docs = [doc for doc in documents if doc.metadata['department'] == dept]
        dept_chunks = []
        
        print(f"📁 {dept}: {len(dept_docs)} docs → ", end="")
        
        for doc in dept_docs:
            # Split the document
            chunks = text_splitter.split_documents([doc])
            
            # Add chunk metadata
            for i, chunk in enumerate(chunks):
                chunk.metadata.update({
                    "chunk_id": f"{doc.metadata['source_file']}_{i}",
                    "chunk_index": i,
                    "total_chunks": len(chunks),
                    "chunk_size": len(chunk.page_content)
                })
            
            dept_chunks.extend(chunks)
            
            # Track stats
            file_type = doc.metadata.get('file_type', 'unknown')
            stats["by_file_type"][file_type] = stats["by_file_type"].get(file_type, 0) + len(chunks)
        
        stats["by_department"][dept] = len(dept_chunks)
        stats["total_chunks"] += len(dept_chunks)
        all_chunks.extend(dept_chunks)
        
        print(f"{len(dept_chunks)} chunks")
    
    print(f"\n🎯 CHUNKING COMPLETE:")
    print(f"   📄 Original: {stats['original_docs']} documents")
    print(f"   ✂️ Created: {stats['total_chunks']} chunks")
    print(f"   📊 Ratio: {stats['total_chunks'] / stats['original_docs']:.1f} chunks per document")
    
    return all_chunks

# Create chunks
chunks = create_smart_chunks(documents)

✂️ Creating smart chunks...
📁 legal_compliance: 28 docs → 80 chunks
📁 business_data: 173 docs → 173 chunks
📁 customer_facing: 5 docs → 105 chunks
📁 internal_operations: 6 docs → 119 chunks

🎯 CHUNKING COMPLETE:
   📄 Original: 212 documents
   ✂️ Created: 477 chunks
   📊 Ratio: 2.2 chunks per document


## Part 2 Complete! 

# Part 3: Vector Embeddings & Search

**Goal**: Transform text chunks into searchable mathematical vectors
**Why Critical**: This enables semantic search - finding meaning, not just keywords
 
## What You'll Learn
- Convert text to numerical vectors (embeddings)
- Build production vector database with FAISS
- Implement semantic search
- Test retrieval accuracy

In [59]:
# Modern imports (no deprecation warnings)
try:
    # Modern approach - no deprecation warnings
    from langchain_huggingface import HuggingFaceEmbeddings
    print("✅ Using modern langchain-huggingface (recommended)")
    modern_import = True
except ImportError:
    # Fallback to deprecated version if needed
    from langchain.embeddings import HuggingFaceEmbeddings
    print("⚠️ Using deprecated import (consider upgrading)")
    print("💡 Run: pip install langchain-huggingface")
    modern_import = False

from langchain.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
import numpy as np
from typing import List, Tuple

print("✅ Vector tools imported!")
print(f"📊 Ready to embed {len(chunks)} chunks from Part 2")

✅ Using modern langchain-huggingface (recommended)
✅ Vector tools imported!
📊 Ready to embed 477 chunks from Part 2


## Setup Embedding Model

**Model Choice**: `all-MiniLM-L6-v2`
- **Fast**: Perfect for development and production
- **Accurate**: Great semantic understanding
- **Compact**: 384 dimensions (vs 1536 for OpenAI)
- **Free**: No API costs

In [60]:
def setup_embedding_model():
    """Initialize the embedding model for vector creation"""
    
    print("🧠 Loading embedding model...")
    
    # Use production-grade embedding model
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    
    # Modern LangChain wrapper (no deprecation warnings)
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={'device': 'cpu'},  # Use CPU for compatibility
        encode_kwargs={'normalize_embeddings': True}  # Better for similarity search
    )
    
    print(f"✅ Model loaded: {model_name}")
    print(f"📐 Vector dimensions: 384")
    print(f"⚡ Device: CPU (production compatible)")
    
    if modern_import:
        print("🎉 Using modern non-deprecated embeddings!")
    
    return embeddings

# Setup embeddings
embeddings = setup_embedding_model()

🧠 Loading embedding model...
✅ Model loaded: sentence-transformers/all-MiniLM-L6-v2
📐 Vector dimensions: 384
⚡ Device: CPU (production compatible)
🎉 Using modern non-deprecated embeddings!


## Create Vector Store

**FAISS**: Facebook's vector search library
- Powers Instagram recommendations
- Billion-scale vector search
- Lightning-fast similarity search
- Industry standard for RAG systems

In [61]:
def create_vector_store(chunks: List, embeddings) -> FAISS:
    """Create FAISS vector store from text chunks"""
    
    print("🔢 Creating vector embeddings...")
    print("⏳ This may take 30-60 seconds...")
    
    # Create vector store with FAISS
    vector_store = FAISS.from_documents(
        documents=chunks,
        embedding=embeddings
    )
    
    print(f"✅ Vector store created!")
    print(f"📊 Vectors: {len(chunks)}")
    print(f"📐 Dimensions: 384 per vector")
    print(f"💾 Total size: ~{len(chunks) * 384 * 4 / 1024 / 1024:.1f} MB")
    
    return vector_store

# Create the vector store
vector_store = create_vector_store(chunks, embeddings)

🔢 Creating vector embeddings...
⏳ This may take 30-60 seconds...
✅ Vector store created!
📊 Vectors: 477
📐 Dimensions: 384 per vector
💾 Total size: ~0.7 MB


## Save Vector Store

In [62]:
# Save vector store for production use
def save_vector_store(vector_store: FAISS, save_path: str = "dataflow_vector_store"):
    """Save vector store to disk for reuse"""
    
    print(f"💾 Saving vector store to '{save_path}'...")
    
    try:
        vector_store.save_local(save_path)
        print(f"✅ Vector store saved successfully!")
        print(f"📁 Location: {save_path}/")
        print(f"🔄 Can be loaded later with: FAISS.load_local('{save_path}', embeddings)")
        return True
    except Exception as e:
        print(f"❌ Save failed: {e}")
        return False

# Save the vector store
save_success = save_vector_store(vector_store)

💾 Saving vector store to 'dataflow_vector_store'...
✅ Vector store saved successfully!
📁 Location: dataflow_vector_store/
🔄 Can be loaded later with: FAISS.load_local('dataflow_vector_store', embeddings)


# Part 4: RAG Agent - Complete Intelligent System
 

In [63]:
# Imports for RAG agent
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage
import time
from typing import Dict, List, Any

# Simple Ollama setup
llm = None

try:
    from langchain.llms import Ollama
    llm = Ollama(model="llama3.2", base_url="http://localhost:11434")
    # Test connection
    test_response = llm.invoke("Hello")
    print("✅ Ollama LLM connected successfully!")
    print("🆓 Using free local LLM")
    print("Hello from Ollama: ", test_response)
    print(f"📊 Vector store ready: {len(chunks)} chunks")
except Exception as e:
    print(f"❌ Ollama connection failed: {e}")
    print("💡 Make sure Ollama is running: ollama serve")
    print("💡 And model is downloaded: ollama pull llama3.2")
    llm = None

✅ Ollama LLM connected successfully!
🆓 Using free local LLM
Hello from Ollama:  How can I assist you today?
📊 Vector store ready: 477 chunks


## Create Customer Service Prompt

In [64]:
# Professional customer service prompt
CUSTOMER_SERVICE_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template="""You are DataFlow's helpful customer service assistant. Your job is to provide accurate, friendly, and professional support to customers.

INSTRUCTIONS:
- Use the provided context to answer questions accurately
- Be concise but thorough in your explanations
- If information isn't in the context, say "I don't have that specific information" and suggest contacting support
- Always maintain a helpful and professional tone
- For technical questions, provide step-by-step guidance when possible

CONTEXT:
{context}

CUSTOMER QUESTION:
{question}

RESPONSE:"""
)

print("✅ Professional customer service prompt created")
print("🎯 Optimized for helpful, accurate responses")

✅ Professional customer service prompt created
🎯 Optimized for helpful, accurate responses


## Build RAG Chain

In [65]:
def create_rag_chain(vector_store, llm, prompt_template):
    """Create production RAG chain"""
    
    if not llm:
        print("❌ No LLM available - cannot create RAG chain")
        print("💡 Please install Ollama or set up OpenAI API key")
        return None
    
    print("🔗 Creating RAG chain...")
    
    # Create retrieval QA chain
    rag_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  # Stuff all context into prompt
        retriever=vector_store.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 4}  # Retrieve top 4 most relevant chunks
        ),
        chain_type_kwargs={
            "prompt": prompt_template
        },
        return_source_documents=True  # Show which documents were used
    )
    
    print("✅ RAG chain created successfully!")
    print("🔍 Retriever: Top 4 most relevant chunks")
    print("🤖 LLM: Ready for customer questions")
    print("📚 Source attribution: Enabled")
    
    return rag_chain

# Create the RAG chain
rag_chain = create_rag_chain(vector_store, llm, CUSTOMER_SERVICE_PROMPT)

🔗 Creating RAG chain...
✅ RAG chain created successfully!
🔍 Retriever: Top 4 most relevant chunks
🤖 LLM: Ready for customer questions
📚 Source attribution: Enabled


## Customer Service Agent

In [66]:
class DataFlowCustomerAgent:
    """Professional customer service agent with simple conversation tracking"""
    
    def __init__(self, rag_chain):
        self.rag_chain = rag_chain
        # Simple conversation tracking (no deprecated memory)
        self.conversation_history = []
        self.conversation_count = 0
        self.response_times = []
        
        print("🤖 DataFlow Customer Service Agent initialized")
    
    def ask(self, question: str) -> Dict[str, Any]:
        """Ask the agent a question and get a comprehensive response"""
        
        if not self.rag_chain:
            return {
                "answer": "I'm sorry, but I'm not properly configured right now. Please contact our support team directly.",
                "sources": [],
                "response_time": 0,
                "error": "No LLM available"
            }
        
        start_time = time.time()
        
        try:
            # Get response from RAG chain
            response = self.rag_chain.invoke({"query": question})
            
            end_time = time.time()
            response_time = end_time - start_time
            
            # Simple conversation tracking
            self.conversation_history.append({
                "question": question,
                "answer": response["result"],
                "timestamp": start_time
            })
            
            # Track metrics
            self.conversation_count += 1
            self.response_times.append(response_time)
            
            # Extract source information
            sources = []
            if "source_documents" in response:
                for doc in response["source_documents"]:
                    sources.append({
                        "department": doc.metadata.get("department", "unknown"),
                        "file": doc.metadata.get("source_file", "unknown"),
                        "preview": doc.page_content[:100] + "..."
                    })
            
            return {
                "answer": response["result"],
                "sources": sources,
                "response_time": response_time,
                "conversation_turn": self.conversation_count
            }
            
        except Exception as e:
            return {
                "answer": f"I apologize, but I encountered an error. Please try rephrasing or contact support.",
                "sources": [],
                "response_time": time.time() - start_time,
                "error": str(e)
            }
    
    def get_stats(self) -> Dict[str, Any]:
        """Get agent performance statistics"""
        
        if not self.response_times:
            return {"conversations": 0, "avg_response_time": 0}
        
        return {
            "conversations": self.conversation_count,
            "avg_response_time": sum(self.response_times) / len(self.response_times),
            "fastest_response": min(self.response_times),
            "slowest_response": max(self.response_times),
            "total_history": len(self.conversation_history)
        }
    
    def get_conversation_history(self, last_n: int = 5):
        """Get recent conversation history"""
        return self.conversation_history[-last_n:] if self.conversation_history else []

# Create the customer service agent
agent = DataFlowCustomerAgent(rag_chain)
print("✅ Customer service agent ready!")

🤖 DataFlow Customer Service Agent initialized
✅ Customer service agent ready!


## Test Customer Scenarios

In [67]:
def test_customer_scenarios(agent):
    """Test agent with realistic customer service scenarios"""
    
    print("🎭 TESTING CUSTOMER SERVICE SCENARIOS")
    print("=" * 45)
    
    # Realistic customer questions
    scenarios = [
        {
            "question": "What are your pricing plans and how much does the premium plan cost?",
            "category": "Billing",
            "expected_dept": "business_data"
        },
        {
            "question": "How do I authenticate with your API? I'm getting authentication errors.",
            "category": "Technical Support",
            "expected_dept": "customer_facing"
        },
        {
            "question": "What data do you collect and how do you protect my privacy?",
            "category": "Privacy/Legal",
            "expected_dept": "legal_compliance"
        }
    ]
    
    results = []
    
    for i, scenario in enumerate(scenarios, 1):
        print(f"\n📞 Scenario {i}: {scenario['category']}")
        print(f"❓ Question: {scenario['question']}")
        print("-" * 50)
        
        # Get agent response
        response = agent.ask(scenario["question"])
        
        print(f"🤖 Agent Response:")
        print(f"   {response['answer']}")  # Show complete response
        
        print(f"\n📚 Sources Used:")
        for j, source in enumerate(response['sources'][:3], 1):  # Show top 3 sources
            print(f"   {j}. 📁 {source['department']} - {source['file']}")
        
        print(f"\n⏱️ Response Time: {response['response_time']:.2f} seconds")
        
        # Check if correct department was used
        dept_match = any(source['department'] == scenario['expected_dept'] for source in response['sources'])
        accuracy = "✅ Accurate" if dept_match else "⚠️ Needs Review"
        print(f"🎯 Department Accuracy: {accuracy}")
        
        results.append({
            "scenario": scenario,
            "response": response,
            "accurate": dept_match
        })
    
    return results

# Test the scenarios
test_results = test_customer_scenarios(agent)

🎭 TESTING CUSTOMER SERVICE SCENARIOS

📞 Scenario 1: Billing
❓ Question: What are your pricing plans and how much does the premium plan cost?
--------------------------------------------------
🤖 Agent Response:
   Our Premium support add-on is part of our Professional Plan, which costs $500/month. This plan includes priority escalation and a dedicated Customer Success Manager (CSM). If you're interested in learning more about our other pricing plans or would like to discuss customization options, I'd be happy to help!

📚 Sources Used:
   1. 📁 business_data - billing_and_pricing.csv
   2. 📁 business_data - billing_and_pricing.csv
   3. 📁 business_data - billing_and_pricing.csv

⏱️ Response Time: 2.54 seconds
🎯 Department Accuracy: ✅ Accurate

📞 Scenario 2: Technical Support
❓ Question: How do I authenticate with your API? I'm getting authentication errors.
--------------------------------------------------
🤖 Agent Response:
   I'd be happy to help you authenticate with our API! There are